In [1]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz
%load_ext line_profiler

from coeditor.common import *
import os
from coeditor.model import RetrievalEditorModel, AttentionMode, DecodingArgs, EditCostModel
from coeditor.service import EditPredictionService, ChangeDetector
from coeditor.c3problem import C3GeneratorCache, C3Problem

os.chdir(proj_root())

In [2]:
# target_dir = proj_root() / "../SPOT-copy/"
target_dir = proj_root()
model_name = "coeditor-xl-c3-dropout-v1.6-resumed"
model_path = get_model_dir(True) / model_name
model = RetrievalEditorModel.load(model_path)
model.to("cuda:2")
detector = ChangeDetector(target_dir)
service = EditPredictionService(
    detector,
    model,
    dec_args = DecodingArgs(do_sample=False, num_beams=1, length_penalty=0.0)
    # dec_args = DecodingArgs(do_sample=True, top_p=0.95, marginalize_samples=50)
)


In [3]:
# service.dec_args = DecodingArgs(do_sample=True, top_p=0.9, marginalize_samples=25)
# service.dec_args = DecodingArgs(do_sample=False, num_beams=8, length_penalty=0.0)
service.tlogger.clear()
response = service.suggest_edit(to_rel_path("scripts/train_model.py"), 91)
print(response)
service.tlogger.as_dataframe()

Target module 'scripts.train_model' has not changed.
Target span has not changed. Creating a trivial change.


Analyzing /home/jiayi/Projects/Coeditor/scripts/train_model.py: 100%|██████████| 230/230 [00:01<00:00, 135.10it/s]


Writing logs to: .coeditor_logs
Target file: /home/jiayi/Projects/Coeditor/scripts/train_model.py
Edit range: (76, 0) - (106, 0)
Target lines: 76--105
	--------------- Suggestion 0 (score: 1) ---------------
	         model = RetrievalEditorModel.from_code_t5("base", reuse_embed=True)
	     else:
	         model = RetrievalEditorModel.load(resumed_from)
 
	     if os.getenv("CUDA_VISIBLE_DEVICES") is None:
	         warnings.warn(
	             "CUDA_VISIBLE_DEVICES not set, using 0. Note that "
	             "the Huggingface Trainer will use all visible GPUs for training."
	         )
	         os.environ["CUDA_VISIBLE_DEVICES"] = "0"
 
	     train_tkn = encoder.edit_tokenizer
	     eval_tkn = copy.deepcopy(train_tkn)
	     eval_tkn.max_query_tks *= 2
	     eval_tkn.max_output_tks *= 2
	     eval_tkn.max_ref_tks_sum *= 2
	+    eval_tkn.max_scope_tks *= 2
 
	     eval_loader = C3DataLoader(
	         datasets["valid"], None, eval_tkn, eval_batch_args, shuffle=False, desc="eval"
	     )

,name,count,avg_time,total_time
1,get c3 problem,1,3.439073,3.439073
0,usage analysis,1,2.789201,2.789201
6,run model,1,1.765540,1.765540
4,model.generate,1,1.759232,1.759232
3,decoder.forward,48,0.014586,0.700150
2,tokenize c3 problem,1,0.050091,0.050091
5,assemble changes,1,0.003297,0.003297
